# HSV変換と肌色抽出

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets

%matplotlib inline

画像を入力して，HSV に変換する

In [ ]:
rgb = cv2.imread("./sample1.png", cv2.IMREAD_COLOR)
if rgb is None:
    print(f'画像が開けませんでした．', file=sys.stderr)
    sys.exit(1)

# OpenCV では BGR の順番であることに注意！
hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)

関数 inRange の引数「**kwargs」の使い方については，

https://dev.classmethod.jp/articles/what-does-asterisk-mean-at-args/

に詳しく解説されています．可変長の辞書型引数です．

In [ ]:
def inRange(**kwargs):
    lower = tuple([int(l) for l, h in kwargs.values()])
    upper = tuple([int(h) for l, h in kwargs.values()])
    binary = cv2.inRange(hsv, lowerb=lower, upperb=upper)

    fig = plt.figure(figsize=(10, 10))
    plt.imshow(binary, cmap='gray')

In [ ]:
# パラメータ lower, upper を設定するスライダー
description = ["hue", "sat.", "val."]
parts = {}
for d in description:
    slider = widgets.SelectionRangeSlider(
        options=np.arange(256), index=(0, 255), description=d
    )
    slider.layout.width = "400px"

    parts[d] = slider

In [ ]:
# ウィジェットを表示する
widgets.interactive(inRange, **parts)

##【補足】

肌色領域の色相（H）と彩度（S）の分布と，ab の分布を表示してみましょう


In [ ]:
# 目視で特定した肌色領域の情報を読み込む
mask = cv2.imread("./sample1_hand_region.tif", cv2.IMREAD_GRAYSCALE)
plt.imshow(mask, cmap='gray')

In [ ]:
hsv_in_mask = hsv[mask==255]
print(type(hsv_in_mask), hsv_in_mask[:,0:2].shape)

In [ ]:
import pandas as pd
import seaborn as sns

df_hs = pd.DataFrame(data=hsv_in_mask[:,0:2], columns=['hue', 'sat.'], dtype='uint8')

sns.set()
sns.set_style("whitegrid")
fig_hs = sns.scatterplot(x='hue', y='sat.', data=df_hs, alpha=0.5)
fig_hs.set(xlim=(-10,190), ylim=(-10,266))

In [ ]:
lab = cv2.cvtColor(rgb, cv2.COLOR_BGR2Lab)
lab_in_mask = lab[mask==255]

df_ab = pd.DataFrame(data=lab_in_mask[:,1:3], columns=['a', 'b'], dtype='uint8')

sns.set()
sns.set_style("whitegrid")
fig_ab = sns.scatterplot(x='a', y='b', data=df_ab, alpha=0.5)
fig_ab.set(xlim=(-10,266), ylim=(-10,266), aspect='equal')